In [19]:
from ipywidgets import VBox, HBox, Layout, interactive_output, FloatSlider
from ipywidgets import interactive as interactive_ipyw

from brian2 import * #type: ignore
import NeuronEquations
import BrianHF
import numpy as np

import matplotlib.pyplot as plt
# NOTE: HAVING NO RESET v=vr CAUSES MORE SPIKES TO OCCUR AS THERE IS NO INTEGRATION TIME NEEDED TO REACH THRESHOLD!!!!!!

In [ ]:
# Assuming NeuronEquations.EQ_SCM_IF and BrianHF.calculate_ChebyshevNeighbours are defined elsewhere
def simulate_neurons_interactive(tau, vt, vr, beta, Wi, Wk, run_time):
    start_scope()
    defaultclock.dt = 0.5*ms

    grid_size_x = 5 # Number of neurons in the x direction
    grid_size_y = 5
    N_Neurons = grid_size_x * grid_size_y # Number of neurons\
        
        
    # Neuron Equation
    Eqs_Neurons = NeuronEquations.EQ_SCM_IF

    # Neuron Parameters
    Neuron_Params = {'tau': tau*ms, 'vt': vt, 'vr': vr, 'P': 0, 'incoming_spikes': 0, 'method': 'exact'}
    # Synapse Parameters
    Syn_Params = {'Num_Neighbours' : 8, 'beta': beta, 'Wi': Wi, 'Wk': Wk}
    Num_Neighbours = Syn_Params['Num_Neighbours']

    input = []
    # Input generation
    for i in range (5):
        input.extend(range(i, N_Neurons, 5))
    times = []
    for i in range(5):
        times.extend([i]*5)
    
    G_Spikes = SpikeGeneratorGroup(N_Neurons, input, times*ms)

    neuronsGrid = NeuronGroup(N_Neurons, Eqs_Neurons+ 'lastspike : second', threshold='v>vt',
                            reset='''
                            lastspike = t
                            v = vr
                            incoming_spikes_post = 0
                            ''',
                            events={'P_ON': 'v > vt', 'P_OFF': '(timestep(t - lastspike, dt) > timestep(dt, dt) and v <= vt)'},
                            namespace=Neuron_Params)
    
    neuronsGrid.set_event_schedule('P_ON', when = 'after_thresholds')
    neuronsGrid.run_on_event('P_ON', 'P = 1' , when = 'after_thresholds')
    neuronsGrid.set_event_schedule('P_OFF', when = 'groups')
    neuronsGrid.run_on_event('P_OFF', 'P = 0', when = 'groups')

    # # Generate x and y values for each neuron
    # x_values = np.repeat(np.arange(grid_size_x), grid_size_y)
    # y_values = np.tile(np.arange(grid_size_y), grid_size_x)

    # # Assign x, y, and z values to each neuron
    # neuronsGrid.X = x_values
    # neuronsGrid.Y = y_values

    # Creating the synapses
    Syn_Input_Neurons = Synapses(G_Spikes, neuronsGrid,
                                 'beta : 1 (constant)',
                                 on_pre='''
                                 ExtIn_post = beta
                                 ''',
                                 namespace=Syn_Params)

    Syn_Neurons_Neurons = Synapses(neuronsGrid, neuronsGrid,
                               '''
                               Wi : 1
                               Wk : 1
                               ''',
                               on_pre={'pre': 'incoming_spikes_post += 1; Exc_pre = Wi',
                                   'pre_2' : 'Inh_post = P_post * (Wk/incoming_spikes_post)'},
                               namespace=Syn_Params)

    # SYNAPSE CONNECTIONS:
    # Connect the first synapses from input to neuronsGrid on a 1 to 1 basis
    Syn_Input_Neurons.connect(j = 'i')
    Syn_Input_Neurons.beta = beta

    # Unzip the pairs into two lists
    indexes_i, indexes_j = BrianHF.calculate_ChebyshevNeighbours(neuronsGrid, Num_Neighbours)
    # Connect the last group of synapses from a neuron to its neighbors
    Syn_Neurons_Neurons.connect(i=indexes_i, j=indexes_j)
    Syn_Neurons_Neurons.Wk = Wk
    Syn_Neurons_Neurons.Wi = Wi

    # SETTING UP MONITORS:
    # Monitor the spikes
    SpikeMon_Input = SpikeMonitor(G_Spikes)
    SpikeMon_Neurons = SpikeMonitor(neuronsGrid)
    
    # RUNNING THE SIMULATION:
    # display(scheduling_summary())
    BrianLogger.log_level_error()
    run(run_time)
    
    return SpikeMon_Input, SpikeMon_Neurons

def update_plot(tau, vt, vr, beta, Wi, Wk):
    run_time = 20*ms
    # Run the simulation with the current slider values
    SpikeMon_Input, SpikeMon_Neurons = simulate_neurons_interactive(tau, vt, vr, beta, Wi, Wk, run_time)
    
    plt.figure(figsize=(10, 6))
    plt.subplot(211)
    plt.plot(SpikeMon_Input.t/ms, SpikeMon_Input.i, '.k')
    plt.xlim(-0.5, run_time/ms+0.5)
    plt.title('Input spikes')
    plt.xlabel('Time (ms)')
    plt.ylabel('Neuron index')
    
    plt.subplot(212)
    plt.plot(SpikeMon_Neurons.t/ms, SpikeMon_Neurons.i, '.k')
    delay_out = +SpikeMon_Neurons.t[0]/ms if len(SpikeMon_Neurons.t) > 0 else 0
    plt.plot(SpikeMon_Input.t/ms+delay_out, SpikeMon_Input.i, 'xr')
    
    plt.legend(['Model Output Spikes', 'Original Input spikes\n + integration delay of {:.2f} ms'.format(delay_out)], loc='upper right')
    plt.xlim(-0.5, run_time/ms+0.5)
    plt.title('Neuron Grid spikes')
    plt.xlabel('Time (ms)')
    plt.ylabel('Neuron index')
    
    plt.tight_layout()

In [20]:


# Create sliders for parameters

tau_slider = FloatSlider(min=0.01, max=5, step=0.01, value=0.2, description='tau', continuous_update=False)
vt_slider = FloatSlider(min=0.01, max=1.0, step=0.01, value=0.1, description='vt', continuous_update=False)
vr_slider = FloatSlider(min=0.0, max=1.1, step=0.1, value=0.0, description='vr', continuous_update=False)
beta_slider = FloatSlider(min=0.1, max=5.0, step=0.05, value=0.5, description='beta', continuous_update=False)
Wi_slider = FloatSlider(min=0.0, max=20.0, step=0.01, value=6.0, description='Wi', continuous_update=False)
Wk_slider = FloatSlider(min=-20.0, max=0.0, step=0.01, value=-3.0, description='Wk', continuous_update=False)

# Display the interactive plot
# Create a dictionary of sliders

# Create a dictionary of sliders
sliders = {
    'tau': tau_slider,
    'vt': vt_slider,
    'vr': vr_slider,
    'beta': beta_slider,
    'Wi': Wi_slider,
    'Wk': Wk_slider
}

# Use the ** operator to unpack the dictionary into keyword arguments
interactive_plot = interactive_ipyw(update_plot, **sliders)
output = interactive_plot.children[-1]
output.layout.height = '600px' # Adjust the height of the output area
# Create a Label widget with vertical space (adjust the number of <br> tags as needed)

# Assuming plot_layout and sliders_layout are defined as before
plot_layout = VBox([output])  # The plot itself is already arranged vertically
sliders_layout = VBox([tau_slider, vt_slider, vr_slider, beta_slider, Wi_slider, Wk_slider])

# Combine the plot and sliders (now with added space) side by side with the sliders on the right
interactive_layout = HBox([plot_layout, sliders_layout])
interactive_layout.layout.align_items = 'center'
# Display the combined layout
display(interactive_layout)

In [21]:
# %matplotlib widget
# import matplotlib.pyplot as plt
# from ipywidgets import HBox, VBox, FloatSlider, interactive_output
# from IPython.display import display

# # Assuming simulate_neurons_interactive function remains the same
# def simulate_neurons_interactive(tau=0.2, vt=0.1, vr=0.0, beta=0.5, Wi=6.0, Wk=-3.0):
#     start_scope()
#     defaultclock.dt = 0.5*ms

#     grid_size_x = 5 # Number of neurons in the x direction
#     grid_size_y = 5
#     N_Neurons = grid_size_x * grid_size_y # Number of neurons\
        
        
#     # Neuron Equation
#     Eqs_Neurons = NeuronEquations.EQ_SCM_IF

#     # Neuron Parameters
#     Neuron_Params = {'tau': tau*ms, 'vt': vt, 'vr': vr, 'P': 0, 'incoming_spikes': 0, 'method': 'exact'}
#     # Synapse Parameters
#     Syn_Params = {'Num_Neighbours' : 8, 'beta': beta, 'Wi': Wi, 'Wk': Wk}
#     Num_Neighbours = Syn_Params['Num_Neighbours']

#     input = []
#     # Input generation
#     for i in range (5):
#         input.extend(range(i, N_Neurons, 5))
#     times = []
#     for i in range(5):
#         times.extend([i]*5)
    
#     G_Spikes = SpikeGeneratorGroup(N_Neurons, input, times*ms)

#     neuronsGrid = NeuronGroup(N_Neurons, Eqs_Neurons+ 'lastspike : second', threshold='v>vt',
#                             reset='''
#                             lastspike = t
#                             incoming_spikes_post = 0
#                             ''',
#                             events={'P_ON': 'v > vt', 'P_OFF': '(timestep(t - lastspike, dt) > timestep(dt, dt) and v <= vt)'},
#                             namespace=Neuron_Params)
    
#     neuronsGrid.set_event_schedule('P_ON', when = 'after_thresholds')
#     neuronsGrid.run_on_event('P_ON', 'P = 1' , when = 'after_thresholds')
#     neuronsGrid.set_event_schedule('P_OFF', when = 'groups')
#     neuronsGrid.run_on_event('P_OFF', 'P = 0', when = 'groups')

#     # # Generate x and y values for each neuron
#     # x_values = np.repeat(np.arange(grid_size_x), grid_size_y)
#     # y_values = np.tile(np.arange(grid_size_y), grid_size_x)

#     # # Assign x, y, and z values to each neuron
#     # neuronsGrid.X = x_values
#     # neuronsGrid.Y = y_values

#     # Creating the synapses
#     Syn_Input_Neurons = Synapses(G_Spikes, neuronsGrid,
#                                  'beta : 1 (constant)',
#                                  on_pre='''
#                                  ExtIn_post = beta
#                                  ''',
#                                  namespace=Syn_Params)

#     Syn_Neurons_Neurons = Synapses(neuronsGrid, neuronsGrid,
#                                '''
#                                Wi : 1
#                                Wk : 1
#                                ''',
#                                on_pre={'pre': 'incoming_spikes_post += 1; Exc_pre = Wi',
#                                    'pre_2' : 'Inh_post = P_post * (Wk/incoming_spikes_post)'},
#                                namespace=Syn_Params)

#     # SYNAPSE CONNECTIONS:
#     # Connect the first synapses from input to neuronsGrid on a 1 to 1 basis
#     Syn_Input_Neurons.connect(j = 'i')
#     Syn_Input_Neurons.beta = beta

#     # Unzip the pairs into two lists
#     indexes_i, indexes_j = BrianHF.calculate_ChebyshevNeighbours(neuronsGrid, Num_Neighbours)
#     # Connect the last group of synapses from a neuron to its neighbors
#     Syn_Neurons_Neurons.connect(i=indexes_i, j=indexes_j)
#     Syn_Neurons_Neurons.Wk = Wk
#     Syn_Neurons_Neurons.Wi = Wi

#     # SETTING UP MONITORS:
#     # Monitor the spikes
#     SpikeMon_Input = SpikeMonitor(G_Spikes)
#     SpikeMon_Neurons = SpikeMonitor(neuronsGrid)
    
#     # RUNNING THE SIMULATION:
#     # display(scheduling_summary())
#     BrianLogger.log_level_error()
#     run_time = 20*ms
#     run(run_time)


#     return SpikeMon_Input, SpikeMon_Neurons

# # Define the update_plot function to use matplotlib for plotting
# def update_plot(tau, vt, vr, beta, Wi, Wk):
#     SpikeMon_Input, SpikeMon_Neurons = simulate_neurons_interactive(tau, vt, vr, beta, Wi, Wk)
    
#     fig, axs = plt.subplots(2, 1, figsize=(10, 6))  # Adjust figsize as needed
    
#     axs[0].plot(SpikeMon_Input.t/ms, SpikeMon_Input.i, '.k')
#     axs[0].set_title('Input spikes')
#     axs[0].set_xlabel('Time (ms)')
#     axs[0].set_ylabel('Neuron index')
    
#     axs[1].plot(SpikeMon_Neurons.t/ms, SpikeMon_Neurons.i, '.k')
#     axs[1].set_title('Neuron Grid spikes')
#     axs[1].set_xlabel('Time (ms)')
#     axs[1].set_ylabel('Neuron index')
    
#     plt.tight_layout()

# # Create sliders
# tau_slider = FloatSlider(min=0.01, max=5, step=0.01, value=0.2, description='tau')
# vt_slider = FloatSlider(min=0.1, max=1.0, step=0.1, value=0.1, description='vt')
# vr_slider = FloatSlider(min=0.0, max=1.1, step=0.1, value=0.0, description='vr')
# beta_slider = FloatSlider(min=0.1, max=5.0, step=0.05, value=0.5, description='beta')
# Wi_slider = FloatSlider(min=0.0, max=20.0, step=0.01, value=6.0, description='Wi')
# Wk_slider = FloatSlider(min=-20.0, max=0.0, step=0.01, value=-3.0, description='Wk')

# # Use interactive_output to create a widget that updates the plot based on the sliders
# interactive_plot = interactive_output(update_plot, {'tau': tau_slider, 'vt': vt_slider, 'vr': vr_slider, 'beta': beta_slider, 'Wi': Wi_slider, 'Wk': Wk_slider})

# # Arrange the sliders vertically
# sliders_layout = VBox([tau_slider, vt_slider, vr_slider, beta_slider, Wi_slider, Wk_slider])

# # Combine the plot and sliders side by side with the sliders on the right
# interactive_layout = HBox([interactive_plot, sliders_layout])
# interactive_layout.layout.align_items = 'center'
# interactive_layout.layout.width = '100%'

# # Display the combined layout
# display(interactive_layout)